In [1]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input,Session

In [2]:
vijay="dettol"#change this variable to uniques name for your bucket
my_region=boto3.session.Session().region_name#set the region of the instance
print(my_region)

us-east-1


In [3]:
s3=boto3.resource('s3')
try:
    if my_region == 'us-east-1':
        s3.create_bucket(Bucket=vijay)
    print('s3 bucket create successfully')
except Exception as e:
    print('S3 error:',e)     

s3 bucket create successfully


In [4]:
# set an output where thr trained model wil be saved
prefix='xgboost-as-a-built-in-algo'
output_path='s3://{}/{}/output'.format(vijay,prefix)
print(output_path)

s3://dettol/xgboost-as-a-built-in-algo/output


# Downloading the dataset and storing in s3

In [5]:
import pandas as pd
import urllib
try:
    urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Success: downloaded bank_clean.csv.')
except Exception as e:
    print('Data load error: ',e)

try:
    model_data = pd.read_csv('./bank_clean.csv',index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)
    

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


In [6]:
#train test split
import numpy as np
train_data,test_data = np.split(model_data.sample(frac=1,random_state =1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


In [7]:
#saving train and test into bucket
#we start train data
import os
pd.concat([train_data['y_yes'],train_data.drop(['y_no','y_yes'],axis=1)],axis=1).to_csv('train.csv',index=False,header=False)
boto3.Session().resource('s3').Bucket(vijay).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(vijay, prefix), content_type='csv')

In [8]:
from sagemaker.session import TrainingInput
#Test Data Into Buckets
pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(vijay).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
validation_input = TrainingInput("s3://{}/{}/{}".format(vijay, prefix, "test/test.csv"), content_type="csv")

# Building model and training data Xgboost-inbuilt Algorithm

In [9]:
#this line automatically looks for the XGboost imagine URI and bilds on XGBoost Container
#specify the repo_version depending on your preference
container=get_image_uri(boto3.Session().region_name,'xgboost',repo_version='1.0-1')

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [10]:
#initialize hyperparameter
hyperparameter={'max_depth':'5',
               'eta':'0.2',
               'gamma':'4',
               'min_child_weight':'6',
               'subsample':'0.7',
               'objective':'binary:logistic',
                'num_round':50
               }

In [11]:

# construct a SageMaker estimator that calls the xgboost-container
estimator = sagemaker.estimator.Estimator(image_uri=container, 
                                          hyperparameters=hyperparameter,
                                          role=sagemaker.get_execution_role(),
                                          train_instance_count=1, 
                                          train_instance_type='ml.m5.2xlarge', 
                                          train_volume_size=5, # 5 GB 
                                          output_path=output_path,
                                          train_use_spot_instances=True,
                                          train_max_run=300,
                                          train_max_wait=600)

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_run has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_use_spot_instances has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_wait has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_volume_size has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [12]:
estimator.fit({'train': s3_input_train,'validation': validation_input})

2021-09-28 14:52:49 Starting - Starting the training job...
2021-09-28 14:53:18 Starting - Launching requested ML instancesProfilerReport-1632840769: InProgress
......
2021-09-28 14:54:16 Starting - Preparing the instances for training............
2021-09-28 14:56:21 Downloading - Downloading input data...
2021-09-28 14:56:50 Training - Training image download completed. Training in progress.
2021-09-28 14:56:50 Uploading - Uploading generated training model.INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
[14:56:44] 28831x5

# Deploy Machine Learning Model

In [13]:
xgb_predictor=estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

-----------------!

In [16]:
from sagemaker.predictor import csv_serializer
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
xgb_predictor.serializer = csv_serializer # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

The csv_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


(12357,)


In [17]:
predictions_array

array([0.05214286, 0.05660191, 0.05096195, ..., 0.03436061, 0.02942475,
       0.03715819])

In [19]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 89.7%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10785)    34% (151)
Purchase        9% (1124)     66% (297) 



# Deleting The Endpoints

In [37]:
#sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(vijay)
bucket_to_delete.objects.all().delete()

[{'ResponseMetadata': {'RequestId': '45E6XXGXC5AJMS0C',
   'HostId': 'S45m/Kv9QiOjxVehbbIfF9RgxgWwslvCiKYF5ThJ8tuQIfG1eaUhRkOTyKmFN5oyQr0sYE13UiQ=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'S45m/Kv9QiOjxVehbbIfF9RgxgWwslvCiKYF5ThJ8tuQIfG1eaUhRkOTyKmFN5oyQr0sYE13UiQ=',
    'x-amz-request-id': '45E6XXGXC5AJMS0C',
    'date': 'Tue, 28 Sep 2021 15:54:21 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2021-09-27-14-46-20-345/rule-output/ProfilerReport-1632753980/profiler-output/profiler-reports/IOBottleneck.json'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2021-09-28-14-52-49-435/output/model.tar.gz'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2021-09-27-14-46-20-345/profiler-output/system/incremental/2021092714/1632754080.algo-1.json'},
   {'Key